<a href="https://colab.research.google.com/github/dykeeIS590DV/dykeeIS590DV.github.io/blob/master/ClimaScope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 📁 Create necessary folders
import os
os.makedirs("data", exist_ok=True)
os.makedirs("plots", exist_ok=True)

# 📦 Install necessary packages
!pip install python-dotenv

# 📚 Import libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import getpass


In [3]:
# 🔐 Securely input NOAA API token
NOAA_TOKEN = getpass.getpass("Enter your NOAA API Token: ")

# Headers for requests
headers = {
    "token": NOAA_TOKEN
}

Enter your NOAA API Token: ··········


In [4]:
# State FIPS codes (lower 48 + AK/HI)
state_fips = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
    '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL',
    '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN',
    '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME',
    '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
    '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH',
    '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
    '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
    '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI',
    '56': 'WY'
}


In [5]:
def fetch_temp_data(state_code, state_abbr):
    url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

    params = {
        "datasetid": "GHCND",
        "datatypeid": "TAVG",
        "locationid": f"FIPS:{state_code}",
        "startdate": "2023-01-01",
        "enddate": "2023-12-31",
        "units": "standard",
        "limit": 1000,
        "offset": 1
    }

    all_data = []
    offset = 1
    max_retries = 3

    while True:
        params["offset"] = offset

        for attempt in range(max_retries):
            response = requests.get(url, headers=headers, params=params)
            code = response.status_code

            if code == 200:
                data = response.json().get("results", [])
                if not data:
                    return pd.DataFrame(all_data)  # no more data
                all_data.extend(data)
                offset += 1000
                time.sleep(1)  # avoid rate limits
                break  # break out of retry loop to continue with next offset

            elif code == 400:
                print(f"❌ Bad Request for {state_abbr}. Check parameters.")
                return pd.DataFrame()

            elif code == 401:
                print(f"🔐 Unauthorized for {state_abbr}. Check API token.")
                raise Exception("Invalid NOAA API token. Aborting all requests.")

            elif code == 429:
                wait_time = 10
                print(f"🚫 Rate limit hit for {state_abbr}. Waiting {wait_time} seconds...")
                time.sleep(wait_time)
                continue  # retry

            elif code >= 500:
                print(f"💥 Server error ({code}) for {state_abbr}. Retrying... ({attempt + 1}/{max_retries})")
                time.sleep(2)
                continue

            else:
                print(f"❗Unexpected error {code} for {state_abbr}. Skipping.")
                return pd.DataFrame()
        else:
            print(f"❌ Max retries reached for {state_abbr}. Moving on.")
            break

    print(f"✅ {state_abbr}: Retrieved {len(all_data)} records.")
    return pd.DataFrame(all_data)

In [ ]:
all_states_data = []

for code, abbr in state_fips.items():
    df = fetch_temp_data(code, abbr)
    df['state'] = abbr
    df.to_csv(f"data/{abbr}_temp_2023.csv", index=False)
    all_states_data.append(df)

# Combine all into one DataFrame
combined_df = pd.concat(all_states_data, ignore_index=True)
combined_df.to_csv("data/all_states_tavg_2023.csv", index=False)

💥 Server error (503) for AL. Retrying... (1/3)
